In [1]:
import warnings
import pandas as pd
from datetime import datetime
from datetime import date
import sqlalchemy as sql
from pathlib import Path
warnings.filterwarnings('ignore')  # Hide warnings
#Library - Project3 
import CryptoDownloadData as coinData
import CryptoPerfSummary as coinAnalytic

#DataBase Tables
# Database connection string
crypto_data_connection_string = 'sqlite:///./Reference/crypto.db'
# Database engine
crypto_data_engine = sql.create_engine(crypto_data_connection_string, echo=True)
'COINBASE_100' #CoinList - symbols
'ETF_LIST' #Sample ETF
'CRYPTO_PX_HISTORY' #Crypto PX History

# If need to repopulate database sqllite, please set refresh = True
refresh = True

In [ ]:
#url = "https://www.ag-grid.com/example-assets/master-detail-data.json"
#df = pd.read_json(url)
#df

data = coinData.get_crypto_px_yf("DPI")
display(data)

In [ ]:
# Download all price history for Coin List 100 into CRYPTO_PX_HISTORY
# If need to rerun, please change refresh = True
if refresh == True:
    coinData.create_coinlist100
    coinData.create_sampleETF
    coinData.drop_table('CRYPTO_PX_HISTORY')
    start_date = datetime(2015, 1, 1)
    end_date = datetime.today()
    coinData.download_px_data_from_COINBASE_100(start_date, end_date)

In [ ]:
# Pull Price by Name        
px_history_df = coinData.get_px_history('WDGLD')
display(px_history_df.head())
display(px_history_df.tail())

In [ ]:
t_date = datetime(2022, 2, 26)
a = coinData.get_market_datas_by_period(t_date)
a

In [2]:
return_matrix = coinData.get_base_pxchanges_matrix()
display(return_matrix)


,A/O Date,Cur PX,1 Day,1 Week,1 Month,3 months,YTD,1 Year,2 Years,3 Years,Start Date,Since Intercept,Start PX,Return
symbol,,,,,,,,,,,,,,
1INCH,2022-02-27,1.396423,-0.025939,-0.119179,-0.140016,-0.618313,-0.448106,-0.689497,NaN,NaN,2021-01-01,-0.689497,4.497286,-0.522522
AAVE,2022-02-27,131.491867,-0.036210,-0.101013,-0.103089,-0.432520,-0.504738,-0.593041,NaN,NaN,2021-01-01,-0.593041,323.108185,-0.625679
ADA,2022-02-27,0.866972,-0.023590,-0.131032,-0.196539,-0.432766,-0.370379,-0.301452,16.531226,19.020592,2021-01-01,-0.301452,1.241105,-0.658071
ALGO,2022-02-27,0.825237,NaN,NaN,NaN,-0.483097,NaN,-0.150367,1.241261,NaN,2021-01-01,-0.150367,0.971286,-0.166391
ALICE,2022-02-27,6.872220,-0.034648,-0.101733,0.025350,-0.656239,-0.485356,NaN,NaN,NaN,2021-01-01,NaN,NaN,-0.643711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XTZ,2022-02-27,3.338848,-0.022761,-0.074138,0.132006,-0.279582,-0.295671,-0.035839,0.205194,7.186661,2021-01-01,-0.035839,3.462956,-0.248305
XYM,2022-02-27,0.142804,-0.013894,-0.094573,-0.109978,-0.584157,-0.361065,NaN,NaN,NaN,2021-01-01,NaN,NaN,0.029604
YFI,2022-02-27,20438.943359,-0.020875,-0.057699,-0.157341,-0.287590,-0.438726,-0.343488,NaN,NaN,2021-01-01,-0.343488,31132.603516,-0.435540


In [ ]:
sql_query = """
SELECT max(date) date FROM CRYPTO_PX_HISTORY order by date desc"""
t_date_str= pd.read_sql_query(sql_query, crypto_data_connection_string)
t_date_str.iat[0,0]
#print(type(t_date))
t_date = datetime.strptime(t_date_str.iat[0,0], '%Y-%m-%d').date()
date_list = coinData.get_market_datas_by_period(t_date)
x_start_date = datetime(2021, 1, 1)
x_end_date = datetime(2021, 6, 1)
y_end_date = t_date 
daily_xy_horizon_return_matrix = coinAnalytic.get_xy_daily_return_matrix(t_date, x_start_date, x_end_date, y_end_date)
display(daily_xy_horizon_return_matrix)

return_matrix = pd.DataFrame()
return_matrix['A/O Date'] = daily_xy_horizon_return_matrix['Y_End Date']
return_matrix['Cur PX'] = daily_xy_horizon_return_matrix['D0']
return_matrix['1 Day'] = (daily_xy_horizon_return_matrix['D0']-daily_xy_horizon_return_matrix['D1'])/daily_xy_horizon_return_matrix['D1']
return_matrix['1 Week'] = (daily_xy_horizon_return_matrix['D0']-daily_xy_horizon_return_matrix['W1'])/daily_xy_horizon_return_matrix['W1']
return_matrix['1 Month'] = (daily_xy_horizon_return_matrix['D0']-daily_xy_horizon_return_matrix['M1'])/daily_xy_horizon_return_matrix['M1']
return_matrix['3 months'] = (daily_xy_horizon_return_matrix['D0']-daily_xy_horizon_return_matrix['M3'])/daily_xy_horizon_return_matrix['M3']
return_matrix['YTD'] = (daily_xy_horizon_return_matrix['D0']-daily_xy_horizon_return_matrix['Y0_YTD'])/daily_xy_horizon_return_matrix['Y0_YTD']
return_matrix['1 Year'] = (daily_xy_horizon_return_matrix['D0']-daily_xy_horizon_return_matrix['Y1'])/daily_xy_horizon_return_matrix['Y1']
return_matrix['2 Years'] = (daily_xy_horizon_return_matrix['D0']-daily_xy_horizon_return_matrix['Y2'])/daily_xy_horizon_return_matrix['Y2']
return_matrix['3 Years'] = (daily_xy_horizon_return_matrix['D0']-daily_xy_horizon_return_matrix['Y3'])/daily_xy_horizon_return_matrix['Y3']
return_matrix['Start Date'] = daily_xy_horizon_return_matrix['Start Date']
return_matrix['Since Intercept'] = (daily_xy_horizon_return_matrix['D0']-daily_xy_horizon_return_matrix['Start Cost'])/daily_xy_horizon_return_matrix['Start Cost']
return_matrix['Start PX'] = daily_xy_horizon_return_matrix['Start Cost']
return_matrix['Return'] = daily_xy_horizon_return_matrix['XY_Return']
return_matrix

#px_change_matrix = daily_xy_horizon_return_matrix['symbol']
#px_change_matrix 
#row_data = daily_xy_horizon_return_matrix.loc[ 'BTC' , : ]
#display(pd.DataFrame(row_data))

In [ ]:
# Get Price History Marix by Period - Sharpe and 30d roll beta, annual return
t_date = datetime(2022, 2, 21)
px_matrix = coinAnalytic.get_crypto_hist_martix_summary(t_date)
display(px_matrix.loc[px_matrix.index.isin(['BTC','ETH','DPI'])])

In [ ]:
#Get SMA, HIGH, LOW, RS RATING
t_date = datetime(2022, 2, 21)
px_strat = coinAnalytic.get_crypto_px_strat(t_date)
display(px_strat)

In [ ]:
a = coinData.get_price_history_by_period(t_date)
display(a)

In [ ]:
t_date = datetime(2022, 2, 24)
x_start_date = datetime(2021, 8, 24)
x_end_date = datetime(2021, 8, 24)
y_end_date = datetime(2022, 2, 24)
daily_xy_horizon_return_matrix = coinAnalytic.get_xy_daily_return_matrix(t_date, x_start_date, x_end_date, y_end_date)
daily_price_matrix = coinAnalytic.get_price_matrix(daily_xy_horizon_return_matrix, x_start_date, y_end_date)
daily_return_matrix = coinAnalytic.get_daily_return_matrix(daily_price_matrix)
cumulative_returns_matrix =  coinAnalytic.get_cumulative_return_matrix(daily_price_matrix)
display(daily_price_matrix[['BTC','ETH','DPI']])

In [ ]:
display(daily_return_matrix[['BTC','ETH','DPI']])

In [ ]:
daily_xy_horizon_return_matrix

In [ ]:
#Our 3 ETF Sample - 
#file_path = Path('./Reference/sampleETF.csv')
#coinData.create_index_from_csv(file_path, 'ETF_LIST')
#coinData.create_sampleETF
sql_query = """
SELECT * FROM ETF_LIST"""
etf_list= pd.read_sql_query(sql_query, crypto_data_connection_string)
etf_summary = etf_list.merge(px_strat, left_on='symbol', right_on='symbol').groupby(['ETF','symbol']).mean().sort_values(by=['ETF','rank'])
etf_summary 

In [ ]:
#3 Portfolios - Return analysis 
perf_summary = etf_list.merge(px_matrix, left_on='symbol', right_on='symbol').groupby(['ETF','symbol']).mean().sort_values(by=['ETF','rank'])
perf_summary 

In [ ]:
file_path = Path('./Reference/sampleETF.csv')
coinData.create_index_from_csv(file_path, 'ETF_LIST')
coinData.create_sampleETF

In [ ]:
file_path = Path('./Reference/Coinbase100.csv')
coinData.create_index_from_csv(file_path, 'COINBASE_100')